In [ ]:
data_set = "v2_data/"

In [9]:
# Load data into dfs
import pandas as pd

# Read CSV files into DataFrames
matches = pd.read_csv(f"{data_set}/matches.csv") # matches with outcome
player_hero_stats = pd.read_csv(f"{data_set}/player_hero_stats_full8.12.25.csv") # each players stats, broken down by hero
player_stats = pd.read_csv(f"{data_set}/player_stats_with_retries.csv") # aggregated player stats as totals (total kills, deaths, matches)
players = pd.read_csv(f"{data_set}/players.csv") # player <> match <> hero data

print(f"Matches csv: \n\n{matches.head()}")
print(f"Player Hero Stats csv: \n\n{player_hero_stats.head()}")
print(f"Player Stats csv: \n\n{player_stats.head()}")
print(f"Players csv: \n\n{players.head()}")

Matches csv: 

   match_id           start_time game_mode match_mode  duration_s winning_team
0  38707530  2025-08-11 05:35:24    Normal   Unranked        1347        Team0
1  38707625  2025-08-11 05:42:39    Normal   Unranked        1988        Team1
2  38707818  2025-08-11 05:58:16    Normal   Unranked        1937        Team0
3  38707829  2025-08-11 05:58:55    Normal   Unranked        1351        Team0
4  38707893  2025-08-11 06:03:43    Normal   Unranked        2012        Team1
Player Hero Stats csv: 

   account_id  hero_id  matches_played  wins  kills  deaths  assists  \
0  1055031308        1              19    10    151     102      200   
1  1055031308        3              19    12    172      80      189   
2  1055031308        4               1     0      3       7        9   
3  1055031308        6             151    89   1336     773     1483   
4  1055031308        7               2     1     13      12       11   

   damage_per_min  time_played  
0      941.842165   

In [10]:
# Merge player stats onto players (which is player+match data)
player_match_stats = pd.merge(players,player_stats, on="account_id",how="left")
print(f"Player Match Stats csv: \n\n{player_match_stats.head()}")

Player Match Stats csv: 

   account_id  match_id   team  hero_id  kills  deaths  assists  denies  \
0  1924441699  38707530  Team0       19      9       2       14       1   
1   124024173  38707530  Team0       15      7       6       21       2   
2  1048356191  38707530  Team1        2      5       4        5       4   
3   381952778  38707530  Team1       31      2       9        2       6   
4  1395432137  38707530  Team1       27      3       7        3       1   

   net_worth winning_team win  matches_played  total_kills  total_deaths  \
0      30541        Team0   Y              34          464           116   
1      28034        Team0   Y             507         4954          3414   
2      23268        Team0   N             465         4388          2327   
3      20278        Team0   N            1062         8414          5956   
4      21881        Team0   N             655         4628          3726   

   total_assists    avg_kd  win_rate  total_time_played  
0       

In [11]:
# rename columns to indicate player stats from match stats
player_match_stats.rename(columns={
    "kills": "pm_kills",
    "deaths": "pm_deaths",
    "assists": "pm_assists",
    "damage_per_min": "pm_damage_per_min",
    "denies": "pm_denies",
    "net_worth": "pm_net_worth",
    "win": "pm_win",
    'matches_played': 'p_total_matches_played',
    'total_kills': 'p_total_kills',
    'total_deaths': 'p_total_deaths',
    'total_assists': 'p_total_assists',
    'avg_kd': 'p_total_avg_kd',
    'win_rate': 'p_total_win_rate',
    'total_time_played': 'p_total_time_played'
},inplace=True)

In [12]:
player_match_stats.to_csv("v2_data/player_match_stats8.12.csv")

In [ ]:
# # build player_hero features, starting with a test set
# test_player_hero = player_hero_stats.copy()

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_player_hero_stats(ph_stats_base: pd.DataFrame) -> pd.DataFrame:
    """
    Create player hero stats by aggregating the player_hero_stats DataFrame.
    Checks for potential divide by zero errors and sets result to 0 if denominator is zero.
    """
    ph_stats = pd.DataFrame()
    ph_stats = ph_stats_base.copy()

    # Avoid divide by zero for deaths
    ph_stats['ph_total_kd'] = np.where(ph_stats['deaths'] == 0, 0, ph_stats['kills'] / ph_stats['deaths'])

    ph_stats['h_total_kd'] = (ph_stats.groupby('hero_id')['ph_total_kd'].transform("mean"))
    # Avoid divide by zero for ph_total_kd
    ph_stats['ph_kd_ratio'] = np.where(ph_stats['ph_total_kd'] == 0, 0, ph_stats['h_total_kd']/ ph_stats['ph_total_kd'])

    ph_stats['h_avg_total_time_played'] = (ph_stats.groupby('hero_id')['time_played'].transform("mean"))
    # Avoid divide by zero for h_avg_total_time_played
    ph_stats['ph_time_played_ratio'] = np.where(ph_stats['h_avg_total_time_played'] == 0, 0, ph_stats['time_played']/ ph_stats['h_avg_total_time_played'])

    ph_stats['h_total_damage_per_min'] = (ph_stats.groupby('hero_id')['damage_per_min'].transform("mean"))
    # Avoid divide by zero for h_total_damage_per_min
    ph_stats['ph_damage_per_min_ratio'] = np.where(ph_stats['h_total_damage_per_min'] == 0, 0, ph_stats['damage_per_min']/ ph_stats['h_total_damage_per_min'])

    ph_stats['h_total_assists'] = (ph_stats.groupby('hero_id')['assists'].transform("mean"))
    # Avoid divide by zero for h_total_assists
    ph_stats['ph_assists_ratio'] = np.where(ph_stats['h_total_assists'] == 0, 0, ph_stats['assists']/ ph_stats['h_total_assists'])

    # Avoid divide by zero for matches_played
    ph_stats['ph_win_rate'] = np.where(ph_stats['matches_played'] == 0, 0, ph_stats['wins'] / ph_stats['matches_played'])
    ph_stats['h_total_win_rate'] = (ph_stats.groupby('hero_id')['ph_win_rate'].transform("mean"))
    # Avoid divide by zero for h_total_win_rate
    ph_stats['ph_win_rate_ratio'] = np.where(ph_stats['h_total_win_rate'] == 0, 0, ph_stats['ph_win_rate'] / ph_stats['h_total_win_rate'])

    ph_stats.rename(columns={
        "wins": "ph_wins",
        "kills": "ph_kills",
        "deaths": "ph_deaths",
        "assists": "ph_assists",
        "damage_per_min": "ph_damage_per_min",
        'time_played': 'ph_time_played'
    }, inplace=True)

    return ph_stats

In [7]:
# testiong different aggregations
# import numpy as np
# def test(df):
#     """
#     Create player hero stats by aggregating the player_hero_stats DataFrame.
#     """


#     return df

# test_player_hero = test(test_player_hero)
# test_player_hero.head(30)

In [14]:
calc_player_hero_stats = create_player_hero_stats(player_hero_stats)
calc_player_hero_stats.head(10)

,account_id,hero_id,matches_played,ph_wins,ph_kills,ph_deaths,ph_assists,ph_damage_per_min,ph_time_played,ph_total_kd,...,ph_kd_ratio,h_avg_total_time_played,ph_time_played_ratio,h_total_damage_per_min,ph_damage_per_min_ratio,h_total_assists,ph_assists_ratio,ph_win_rate,h_total_win_rate,ph_win_rate_ratio
0,1055031308,1,19,10,151,102,200,941.842165,37434,1.480392,...,0.796885,108048.259449,0.346456,863.109847,1.091219,523.382641,0.382130,0.526316,0.513354,1.025249
1,1055031308,3,19,12,172,80,189,898.590489,34818,2.150000,...,0.760218,81237.989595,0.428593,889.281800,1.010468,467.748083,0.404064,0.631579,0.503753,1.253748
2,1055031308,4,1,0,3,7,9,871.488912,1894,0.428571,...,2.876172,83179.213933,0.022770,1046.757534,0.832560,450.958310,0.019957,0.000000,0.523961,0.000000
3,1055031308,6,151,89,1336,773,1483,803.349733,283198,1.728331,...,0.658813,95625.943662,2.961518,786.509776,1.021411,513.984290,2.885302,0.589404,0.538333,1.094869
4,1055031308,7,2,1,13,12,11,919.599479,4093,1.083333,...,1.391383,98864.750000,0.041400,855.699266,1.074676,461.608350,0.023830,0.500000,0.541796,0.922857
5,1055031308,13,16,9,124,73,152,899.036713,28203,1.698630,...,0.871668,131337.462745,0.214737,871.381058,1.031738,578.842157,0.262593,0.562500,0.520137,1.081447
6,1055031308,14,16,4,121,90,196,871.037350,34075,1.344444,...,0.951273,59645.157864,0.571295,753.474086,1.156028,381.112760,0.514283,0.250000,0.478226,0.522765
7,1055031308,15,5,2,30,40,49,757.284160,12159,0.750000,...,1.447530,121811.289911,0.099818,796.424577,0.950855,696.319290,0.070370,0.400000,0.480504,0.832460
8,1055031308,16,83,48,795,427,1025,1027.602254,160168,1.861827,...,0.856724,72951.881702,2.195529,886.162972,1.159609,404.593823,2.533405,0.578313,0.541543,1.067899
9,1055031308,19,21,6,174,136,141,912.358626,41387,1.279412,...,1.111204,117060.163737,0.353553,836.812569,1.090278,493.871924,0.285499,0.285714,0.483297,0.591177


In [11]:
calc_player_hero_stats.describe()

c:\Code\Local Code\deadlock_match_prediction\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,account_id,hero_id,matches_played,ph_wins,ph_kills,ph_deaths,ph_assists,ph_damage_per_min,ph_time_played,ph_total_kd,...,ph_kd_ratio,h_avg_total_time_played,ph_time_played_ratio,h_total_damage_per_min,ph_damage_per_min_ratio,h_total_assists,ph_assists_ratio,ph_win_rate,h_total_win_rate,ph_win_rate_ratio
count,4.650000e+04,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,4.650000e+04,46500.000000,...,4.650000e+04,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000,46500.000000
mean,7.343405e+08,19.853226,47.967892,25.739591,362.294086,295.402194,504.321011,841.317376,9.157209e+04,1.272859,...,inf,91572.086710,1.000000,841.317376,1.000000,504.321011,1.000000,0.510694,0.510694,1.000000
std,6.248374e+08,16.619676,109.768951,60.058353,859.711857,681.436526,1169.632594,236.592509,2.098881e+05,1.121935,...,NaN,26378.778148,2.305225,97.100351,0.261021,156.498376,2.282105,0.239424,0.027500,0.469850
min,1.776180e+05,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.950000e+02,0.000000,...,2.413847e-02,39417.635394,0.002399,601.899229,0.000000,193.848254,0.000000,0.000000,0.468608,0.000000
25%,1.701789e+08,7.000000,4.000000,2.000000,26.000000,26.000000,41.000000,693.478549,7.772750e+03,0.763636,...,8.692605e-01,79136.957027,0.091670,775.496568,0.847630,450.958310,0.088754,0.416667,0.483297,0.821333
50%,4.237003e+08,15.000000,15.000000,8.000000,102.000000,92.000000,152.000000,832.469327,2.837750e+04,1.086485,...,1.162279e+00,91503.122796,0.323097,851.532695,0.984503,493.871924,0.318544,0.511716,0.513354,1.018870
75%,1.179052e+09,27.000000,45.000000,24.000000,324.000000,280.000000,468.000000,973.233386,8.532900e+04,1.500000,...,1.588944e+00,104324.853685,0.947382,889.281800,1.131566,562.511246,0.951390,0.615385,0.524026,1.193723
max,1.926423e+09,60.000000,3078.000000,1586.000000,23428.000000,21072.000000,38150.000000,3027.155812,5.644578e+06,43.000000,...,inf,168409.298087,57.317675,1046.757534,3.235306,1047.444740,62.247644,1.000000,0.570825,2.133982


In [15]:
# join player_match_stats on player_hero_stats on account_id AND hero_id
phm_stats = pd.merge(player_match_stats,calc_player_hero_stats,on=["hero_id", "account_id"], how="left")
phm_stats.head(11)

,account_id,match_id,team,hero_id,pm_kills,pm_deaths,pm_assists,pm_denies,pm_net_worth,winning_team,...,ph_kd_ratio,h_avg_total_time_played,ph_time_played_ratio,h_total_damage_per_min,ph_damage_per_min_ratio,h_total_assists,ph_assists_ratio,ph_win_rate,h_total_win_rate,ph_win_rate_ratio
0,1924441699,38707530,Team0,19,9,2,14,1,30541,Team0,...,0.317067,117060.163737,0.158192,836.812569,1.211493,493.871924,0.234879,0.818182,0.483297,1.692916
1,124024173,38707530,Team0,15,7,6,21,2,28034,Team0,...,0.812438,121811.289911,0.227869,796.424577,1.083957,696.319290,0.231216,0.562500,0.480504,1.170646
2,1048356191,38707530,Team1,2,5,4,5,4,23268,Team0,...,0.599448,97638.637427,2.205193,944.072300,1.278855,560.819688,2.141508,0.675439,0.570825,1.183267
3,381952778,38707530,Team1,31,2,9,2,6,20278,Team0,...,0.943616,168409.298087,7.170412,842.696537,0.921388,1047.444740,5.925850,0.541864,0.508762,1.065065
4,1395432137,38707530,Team1,27,3,7,3,1,21881,Team0,...,0.999597,104324.853685,0.470933,860.976855,1.116680,535.128026,0.540058,0.448276,0.508797,0.881051
5,1902828658,38707530,Team0,25,10,4,11,3,28896,Team0,...,1.038823,92429.599897,0.431215,958.547073,1.200750,468.858757,0.362583,0.458333,0.559999,0.818453
6,1482595800,38707530,Team1,11,1,10,5,2,20134,Team0,...,1.294742,83956.425760,5.284872,601.899229,0.751969,587.051282,5.962000,0.616379,0.515569,1.195533
7,1176357514,38707530,Team0,13,3,5,15,4,28034,Team0,...,1.746398,131337.462745,0.479421,871.381058,0.841847,578.842157,0.568376,0.485714,0.520137,0.933821
8,192227354,38707530,Team0,52,10,3,12,4,29669,Team0,...,0.640459,79136.957027,0.572792,935.662883,1.447118,450.301314,0.621806,0.538462,0.514793,1.045978
9,1730032433,38707530,Team0,14,4,2,13,6,28332,Team0,...,0.653369,59645.157864,0.300309,753.474086,1.172641,381.112760,0.375217,0.500000,0.478226,1.045530


In [16]:
phm_stats.to_csv("v2_data/phm_stats.8.12_v2.csv")

Quantile training data

In [ ]:
def q25(x): return x.quantile(0.25)
def q75(x): return x.quantile(0.75)

def create_quantile_team_stats(phm_stats_base: pd.DataFrame) -> pd.DataFrame:
    """
    Create team-level stats based on player&player_hero stats in min, max, 25%, median, and 75% quantiles
    """
    phm_stats = pd.DataFrame()
    phm_stats = phm_stats_base.copy()

    # set the team stats to be set to quantiles
    team_stats = [
        'p_total_time_played',
        'ph_wins','ph_kills','ph_deaths','ph_assists',
        'ph_damage_per_min','ph_time_played','ph_total_kd',
        'ph_kd_ratio', 'ph_time_played_ratio',
        'ph_damage_per_min_ratio','ph_assists_ratio','ph_win_rate_ratio'
    ]

    # check for missing columns
    missing_cols = [col for col in team_stats if col not in phm_stats.columns]
    if missing_cols:
        print(f"*CRITICAL* Missing columns in team stats: {missing_cols}")
        return pd.DataFrame()  # Return an empty DataFrame if missing columns are found

    # for each columm, set min, max, and quantiles
    agg_function = {
        col: ["min", "max",
              q25,
              "median",
              q75]
        for col in team_stats
    }

    agg_function['pm_win'] = 'first'
    
    # group by columns, using agg_function as the aggregation function
    phm_stats = (phm_stats.groupby(
        ['match_id','team']).agg(
            agg_function))    
    
    # converts  tuples into col:val pairs
    def clean_columns(c_tuple):
        """converts the tuples from lambda into strings"""
        col, stat = c_tuple
        if stat == "min":
            return f"{col}_min"
        elif stat == "max":
            return f"{col}_max"
        elif stat == "median":
            return f"{col}_median"
        elif stat == "q25":
            return f"{col}_q25"
        elif stat == "q75":
            return f"{col}_q75"
        elif callable(stat):
            return f"{col}_{stat.__name__}"
        else:
            return f"{col}_{stat}"

    phm_stats.columns = [clean_columns(col) for col in phm_stats.columns]

    return phm_stats.reset_index()


Run quantile data processing (V1)

In [40]:
from datetime import datetime

now = datetime.now()
date = now.date()
hour = now.time()

team_stats = create_quantile_team_stats(phm_stats)
team_stats.to_csv(f"v2_data/team_stats_qunatiles_{date}.csv")

STD training data

In [28]:

def create_std_team_stats(phm_stats_base: pd.DataFrame) -> pd.DataFrame:
    """
    Create team-level stats based on player&player_hero stats in min, max, mean, and std.
    """

    phm_stats = pd.DataFrame()
    phm_stats = phm_stats_base.copy()

    # set the team stats to be set to quantiles
    team_stats = [
        'p_total_time_played',
        'ph_wins','ph_kills','ph_deaths','ph_assists',
        'ph_damage_per_min','ph_time_played','ph_total_kd',
        'ph_kd_ratio', 'ph_time_played_ratio',
        'ph_damage_per_min_ratio','ph_assists_ratio','ph_win_rate_ratio'
    ]

    # check for missing columns
    missing_cols = [col for col in team_stats if col not in phm_stats.columns]
    if missing_cols:
        print(f"*CRITICAL* Missing columns in team stats: {missing_cols}")
        return pd.DataFrame()  # Return an empty DataFrame if missing columns are found

    # for each columm, set min, max, and quantiles
    agg_function = {
        col: ["min", "max",
                "mean",
                "std"]
        for col in team_stats
    }

    agg_function['pm_win'] = 'first'
    
    # group by columns, using agg_function as the aggregation function
    phm_stats = (phm_stats.groupby(
        ['match_id','team']).agg(
            agg_function))    
    
    # converts lambda tuples into col:val pairs
    def clean_columns(c_tuple):
        col, stat = c_tuple
        if stat == "min":
            return f"{col}_min"
        elif stat == "max":
            return f"{col}_max"
        elif stat == "mean":
            return f"{col}_mean"
        elif stat == "std":
            return f"{col}_std"
        elif callable(stat):
            return f"{col}_{stat.__name__}"
        else:
            return f"{col}_{stat}"

    phm_stats.columns = [clean_columns(col) for col in phm_stats.columns]

    return phm_stats.reset_index()

Run standard deviation based processing (V2)

In [29]:
from datetime import datetime

now = datetime.now()
date = now.date()
hour = now.time()

team_stats = create_std_team_stats(phm_stats)
team_stats.to_csv(f"v2_data/team_stats_std_{date}.csv")

Checkpoint, load team_stats and run data processing if needed.

In [ ]:
# team_stats = pd.read_csv(f"v2_data/team_stats_{date}.csv")

Pivot data into single row per match, each stat has Team0 and Team1

In [30]:
def create_training_data(team_stat_base:pd.DataFrame) -> pd.DataFrame:
    """
    Create training data by merging team stats with match outcomes.
    """
    t_stats = team_stat_base.copy()
    t_stats = t_stats.pivot(index='match_id', columns='team')

    t_stats.columns = [f'{col[0]}_{col[1]}' for col in t_stats.columns]
    t_stats['team_0_win'] = t_stats['pm_win_first_Team0']
    t_stats.drop('pm_win_first_Team0', axis=1, inplace=True)
    t_stats.drop('pm_win_first_Team1', axis=1, inplace=True)
    t_stats = t_stats.reset_index()

    return t_stats

In [31]:
training_data = create_training_data(team_stats)
training_data.to_csv(f"v2_data/training_data_{date}.csv")
training_data.head(25)

,match_id,p_total_time_played_min_Team0,p_total_time_played_min_Team1,p_total_time_played_max_Team0,p_total_time_played_max_Team1,p_total_time_played_mean_Team0,p_total_time_played_mean_Team1,p_total_time_played_std_Team0,p_total_time_played_std_Team1,ph_wins_min_Team0,...,ph_assists_ratio_std_Team1,ph_win_rate_ratio_min_Team0,ph_win_rate_ratio_min_Team1,ph_win_rate_ratio_max_Team0,ph_win_rate_ratio_max_Team1,ph_win_rate_ratio_mean_Team0,ph_win_rate_ratio_mean_Team1,ph_win_rate_ratio_std_Team0,ph_win_rate_ratio_std_Team1,team_0_win
0,38648445,370484,452801,1618549,2333503,8.429413e+05,9.871765e+05,4.801396e+05,7.022480e+05,2,...,0.691545,1.016011,1.022296,1.936208,1.397729,1.257881,1.225829,0.338350,0.153874,Y
1,38648734,730576,682674,3542797,3095841,2.366166e+06,1.703761e+06,9.558800e+05,8.744691e+05,20,...,6.844289,1.039758,0.492014,1.369036,1.358353,1.126812,1.012133,0.125570,0.286544,Y
2,38648877,337492,217922,6354509,4199569,2.307951e+06,2.138568e+06,2.146047e+06,1.351296e+06,8,...,10.528364,0.838749,0.965532,1.233189,1.116964,1.033349,1.046715,0.144230,0.063422,N
3,38649032,412456,215022,2368830,4004780,1.370147e+06,1.883512e+06,6.946286e+05,1.236927e+06,5,...,8.054461,0.636101,0.823960,1.159298,1.272819,1.008722,1.075956,0.198922,0.175792,N
4,38649125,1502187,1304533,2423327,3296440,2.111556e+06,2.366959e+06,3.490991e+05,8.335851e+05,49,...,7.245584,1.037910,0.969101,1.300616,1.192636,1.165100,1.120919,0.094979,0.080252,N
5,38649416,1203665,679614,6354509,3542797,3.220325e+06,2.357672e+06,1.858969e+06,9.733862e+05,38,...,2.626361,0.821189,0.914635,1.295020,1.502969,1.080557,1.102309,0.161774,0.213506,N
6,38649760,478869,1629895,4524706,4676863,2.608683e+06,2.791120e+06,1.572898e+06,1.284924e+06,13,...,19.869841,0.947748,0.926567,1.222662,1.561339,1.065328,1.152391,0.102432,0.216416,Y
7,38649994,679614,1245849,2044728,2484181,1.290074e+06,1.932430e+06,5.730772e+05,5.461696e+05,16,...,5.974570,0.986409,0.888700,1.307380,1.077202,1.121146,1.008200,0.134039,0.068184,Y
8,38649996,1203665,124316,4199569,4097620,2.072858e+06,1.542705e+06,1.150482e+06,1.515054e+06,27,...,4.476783,0.802729,0.844651,1.295020,1.191512,1.073966,1.037345,0.177890,0.115652,Y
9,38650514,518824,1452430,3582803,5553117,1.550105e+06,2.740627e+06,1.247447e+06,1.450274e+06,15,...,1.198887,0.944993,0.920787,1.404045,1.100381,1.137777,1.035176,0.148363,0.068847,N


Pivot data into single role per match, each stat is a differential

In [41]:
def create_differential_training_data(team_stat_base:pd.DataFrame) -> pd.DataFrame:
    """
    Create training data by merging team stats with match outcomes.
    Each stat is a differential: Team0 - Team1
    """
    t_stats = team_stat_base.copy()
    t_stats = t_stats.pivot(index='match_id', columns='team')

    t_stats.columns = [f'{col[0]}_{col[1]}' for col in t_stats.columns]
    t_stats['team_0_win'] = t_stats['pm_win_first_Team0']
    t_stats.drop('pm_win_first_Team0', axis=1, inplace=True)
    t_stats.drop('pm_win_first_Team1', axis=1, inplace=True)
    
    # create differential columns
    for col in t_stats.columns:
        if col.endswith('_Team0'):
            base_col = col[:-6]  # remove '_Team0' to get the base column name
            team1_col = f'{base_col}_Team1'
            if team1_col in t_stats.columns:
                t_stats[f'{base_col}_diff'] = (t_stats[col] - t_stats[team1_col]).round(3)
                t_stats.drop(col, axis=1, inplace=True)
                t_stats.drop(team1_col, axis=1, inplace=True)

    t_stats = t_stats.reset_index()

    return t_stats

In [42]:
training_data = create_differential_training_data(team_stats)
training_data.to_csv(f"v2_data/training_data_differentials_{date}.csv")
training_data.head(25)

,match_id,team_0_win,p_total_time_played_min_diff,p_total_time_played_max_diff,p_total_time_played_q25_diff,p_total_time_played_median_diff,p_total_time_played_q75_diff,ph_wins_min_diff,ph_wins_max_diff,ph_wins_q25_diff,...,ph_assists_ratio_min_diff,ph_assists_ratio_max_diff,ph_assists_ratio_q25_diff,ph_assists_ratio_median_diff,ph_assists_ratio_q75_diff,ph_win_rate_ratio_min_diff,ph_win_rate_ratio_max_diff,ph_win_rate_ratio_q25_diff,ph_win_rate_ratio_median_diff,ph_win_rate_ratio_q75_diff
0,38648445,Y,-82317,-714954,-63787.25,-15897.5,28579.75,-10,352,-0.75,...,-0.343,17.151,0.078,0.957,4.969,-0.006,0.538,0.004,-0.113,-0.146
1,38648734,Y,47902,446956,962662.25,991897.5,655119.25,16,-8,-35.75,...,0.441,-6.585,-1.115,-2.702,-7.955,0.548,0.011,0.062,0.020,0.015
2,38648877,N,119570,2154940,-425985.25,-365769.0,-151904.00,-5,-166,-15.75,...,-0.234,-12.813,-0.275,-1.402,-13.830,-0.127,0.116,-0.023,-0.057,0.028
3,38649032,N,197434,-1635950,-526148.25,-312911.5,-392193.50,-1,-263,-7.75,...,-0.125,-10.065,-0.157,0.386,-6.454,-0.188,-0.114,0.013,-0.014,-0.075
4,38649125,N,197654,-873113,333224.75,-448923.0,-527019.50,-13,-122,35.75,...,-0.454,4.199,2.505,3.754,3.974,0.069,0.108,-0.018,0.051,0.033
5,38649416,N,524051,2811712,-62467.50,249379.0,1193201.75,28,250,40.00,...,0.990,9.925,1.789,1.821,3.307,-0.093,-0.208,0.040,0.061,0.028
6,38649760,Y,-1151026,-152157,-435201.75,661861.5,-70242.75,5,16,1.00,...,0.047,-2.639,0.432,4.167,-2.484,0.021,-0.339,-0.063,-0.062,-0.028
7,38649994,Y,-566235,-439453,-559477.00,-980408.5,-596452.75,-18,225,-29.25,...,-0.957,-3.566,-1.170,-1.425,-7.982,0.098,0.230,0.021,0.074,0.167
8,38649996,Y,1079349,101949,830087.75,578154.0,91469.50,25,7,27.00,...,1.071,0.660,0.824,1.150,-0.221,-0.042,0.104,-0.040,0.071,0.088
9,38650514,N,-933606,-1970314,-1374917.25,-1362452.0,-442039.00,-40,237,-36.25,...,-1.392,2.273,-1.550,0.066,3.075,0.024,0.304,0.106,0.058,0.052
